In [2]:
import pandas as pd
import numpy as np
from itertools import combinations, product, chain, repeat
import copy

In [8]:
Rating = pd.read_csv('ExpData/01_蕭柏圓/1_VBDM_2022-12-27-2049_Rating.csv')
Rating = Rating.sort_values(
    'Name', key=lambda x: sorted(x.str.slice(start=2, stop=-4)))
Rating.loc[Rating.query('rating==10').index, 'rating'] = 9
ratings_counts = Rating['rating'].value_counts()
ratings_counts

2.0    78
3.0    57
5.0    41
6.0    40
7.0    38
4.0    32
1.0    32
Name: rating, dtype: int64

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations, product, chain, repeat
import copy

def genCombinations(countList):
    temp = []
    i = 4
    for count in countList:
        i += 1
        temp.append(np.repeat(i,count))

    ratings = []
    for i in range(len(temp)):
        stimRating = temp[i]
        ratings.append([str(j)+'_'+str(i+1) for i,j in enumerate(stimRating)])

    diffList = [[] for i in range(4)]
    for c in list(combinations(ratings,2)):
        diff = abs(int(c[1][0].split('_')[0])-int(c[0][0].split('_')[0]))
        diffList[diff-1].append(c)
        
    RatingCombinations = []
    diffCombinations =[[] for i in range(4)]
    i=0
    for diff in diffList:
        for rate in diff:
            this = list(product(rate[0], rate[1]))
            RatingCombinations += this
            diffCombinations[i]+= this
        i+=1
    # allcom = np.array(RatingCombinations).flatten()
    
    return diffCombinations, RatingCombinations, ratings

In [ ]:
def getStim(diff, diffCombinations, diffComcopy, stimList, quotaDF):
    def checkQuota(stim):
        if (quotaDF[quotaDF['Name']==stim[0]]['NowCounts'].values[0]==0) or \
            (quotaDF[quotaDF['Name']==stim[1]]['NowCounts'].values[0]==0):
            return False
        return True
    global failed
    ind = diff-1
    # needCounts = [150, 120, 90, 60]
    needCounts = [160, 120, 80, 40]
    for i in range(len(diffCombinations[ind])):
        if len(stimList[ind]) == needCounts[ind]:
            break
        index = np.random.randint(len(diffCombinations[ind]))
        toAdd = diffCombinations[ind][index]
        diffCombinations[ind].pop(index)
        if checkQuota(toAdd):
            stimList[ind].append(toAdd)
            diffComcopy[ind].remove(toAdd)
            for img in toAdd:
                updateIndex = quotaDF[quotaDF['Name']==img].index[0]
                quotaDF.at[updateIndex, 'NowCounts'] -= 1
        else:
            failed+=1

In [ ]:
failed = 0
def createFirstStim(NowCounts, allcounts):
    print(NowCounts)
    diffCombinations, RatingCombinations, ratings = genCombinations(allcounts) # [6,8,10,10,8,12]
    diffComcopy = copy.deepcopy(diffCombinations)
    stimList = [[] for i in range(4)]
    allratings = [item for rating in ratings for item in rating]
    quotaDF = pd.DataFrame({'Name':allratings, 'NowCounts': NowCounts})
    global failed
    failed = 0
    for i in [4,3,2,1]:
        getStim(i, diffCombinations, diffComcopy, stimList, quotaDF)
    stimOccur = []
    for ind in range(4):
        stimOccur += list(sum(stimList[ind], ()))
    print(len([item for rating in stimList for item in rating]))
    return stimList, stimOccur, diffComcopy

In [ ]:
def createSecondStim(NowCounts, prevDiffCom, allcounts):
    print(NowCounts)
    diffComcopy = copy.deepcopy(prevDiffCom)
    stimList = [[] for i in range(4)]
    __, _, ratings = genCombinations(allcounts) # [6,8,10,10,8,12]
    allratings = [item for rating in ratings for item in rating]
    quotaDF = pd.DataFrame({'Name':allratings, 'NowCounts': NowCounts})
    global failed
    failed = 0
    for i in [4,3,2,1]:
        getStim(i, prevDiffCom, diffComcopy, stimList, quotaDF)
    stimOccur = []
    for ind in range(4):
        stimOccur += list(sum(stimList[ind], ()))
    print(len([item for rating in stimList for item in rating]))
    return stimList, stimOccur, diffComcopy

In [2]:
def makeStim(allcounts):
    NowCounts=3
    while True:
        stimList1, stimOccur1, newDiffComcopy = createFirstStim(NowCounts, allcounts)
        total = len([item for rating in stimList1 for item in rating])
        if total == 400:
            break
        else:
            NowCounts+=1
    NowCounts=3
    while True:
        diffComcopy = copy.deepcopy(newDiffComcopy)
        stimList2, stimOccur2, lastDiffComcopy = createSecondStim(NowCounts, diffComcopy, allcounts)
        total = len([item for rating in stimList2 for item in rating])
        if total == 400:
            break
        else:
            NowCounts+=1
    
    return stimList1, stimList2
    

In [14]:
a, b = makeStim([50,30,20,20,16])

5
302
6
398
7
420
5
294
6
396
7
420


In [31]:
stimOccur = []
for ind in range(4):
    stimOccur += a[ind]
stimOccur2 = []
for ind in range(4):
    stimOccur2 += b[ind]

In [32]:
stimOccur = set(stimOccur)
stimOccur2 = set(stimOccur2)

In [34]:
new = stimOccur.union(stimOccur2)
print(len(stimOccur))
print(len(stimOccur2))
print(len(new))

420
420
840


In [ ]:
hard = stimList1[0]
highLowList = [0 for i in range(4)]
for stim in hard:
    first = int(stim[0].split('_')[0])
    second = int(stim[1].split('_')[0])
    match first:
        case 5:
            highLowList[0]+=1
        case 6:
            highLowList[1]+=1
        case 7:
            highLowList[2]+=1
        case 8:
            highLowList[3]+=1
print(highLowList)

In [ ]:
unique, counts = np.unique(stimOccur1, return_counts=True)
stim1df = pd.DataFrame({'Name': unique, 'counts': counts})
stim1df[stim1df.counts < 7].reset_index()

In [ ]:
unique, counts = np.unique(stimOccur2, return_counts=True)
stim2df = pd.DataFrame({'Name': unique, 'counts': counts})
stim2df[stim2df.counts < 7].reset_index()